In [1]:
import pandas as pd
import numpy as np
import time

In [23]:
def items2vec(_df_item,_session_items):
    '''
    input1: item df
    input2: list of item_id in session
    '''
    session_vec = np.sum(_df_item.loc[pd.Index(_session_items)]).values
    return session_vec 

In [14]:
def sessions2vec(item_vec_df,_df_session):
    '''
    input 1 dataframe with col name :***item_id as index***
    input 2 session.csv df with col session_id and item_id
    return df with sessions as row, features as columns
    '''
    #item_vec裡面需要有item_id
    s_t = time.time()
    _df_item = item_vec_df
    pre_sid = None
    _session_dic={}
    item_buf=[]
    for row in _df_session.iterrows():#tuple of row value,row[0]=index,row[1]=values
        if pre_sid!=row[1].session_id:#check if session id matches
            if (item_buf):#if value in buf
                _session_dic[pre_sid] = (items2vec(_df_item,item_buf))#calculate sum vec for session
            pre_sid = row[1].session_id# move to next session
            item_buf=[]
        item_buf.append(row[1].item_id)
    _session_dic[pre_sid] = (items2vec(_df_item,item_buf))#saving last session vec 
    e_t=time.time()
    print("total session: ",len(_session_dic)," time usage: ",e_t-s_t," sec")
    re_df = pd.DataFrame(_session_dic).T
    re_df = re_df.rename({idx:elem for idx,elem in enumerate(item_vec.columns)}, axis='columns')
    return re_df

In [2]:
item_vec = pd.read_csv("../datasets/item_no_n_onehot.csv",index_col=0)
item_vec.set_index("item_id",inplace=True)
item_vec.head()

,10_147,10_159,10_184,10_217,10_22,10_287,10_361,10_407,10_464,10_561,...,68_1,69_1,6_1,70_1,71_1,72_1,73_1,7_1,8_1,9_1
item_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.336008,1.071298,-0.264768,-0.172988,-0.145533,-0.423388,-0.987598,-0.117550,-0.278755,-0.072062
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.746646,-0.001409,-0.264768,-0.172988,-0.145533,-0.423388,1.072686,0.080247,-0.278755,-0.072062
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.243919,-0.199109,-0.264768,-0.172988,-0.145533,-0.423388,1.072686,1.393206,-0.278755,-0.072062
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.288110,-0.001409,-0.264768,-0.172988,-0.145533,-0.423388,-0.987598,0.366711,-0.278755,-0.072062
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.336008,-0.001409,-0.264768,-0.172988,-0.145533,-0.423388,1.072686,1.260205,-0.278755,-0.072062


### turn train sessions to vector

In [10]:
sessions_train_x = pd.read_csv("../datasets/train_sessions_x.csv",index_col=0)
sessions_test_x = pd.read_csv("../datasets/test_sessions_x.csv",index_col=0)
sessions_test_x.head()

,session_id,item_id,date
93,113,2171,2021-05-05 13:19:15.147
94,113,17218,2021-05-05 14:13:21.645
95,113,15738,2021-05-05 13:19:54.211
96,113,19760,2021-05-05 13:18:49.495
97,113,21215,2021-05-05 14:03:43.401


In [15]:
session_train = sessions2vec(item_vec,sessions_train_x)
session_test = sessions2vec(item_vec,sessions_test_x)
session_test

total session:  918382  time usage:  490.6493601799011  sec
total session:  81618  time usage:  47.05442571640015  sec


,10_147,10_159,10_184,10_217,10_22,10_287,10_361,10_407,10_464,10_561,...,68_1,69_1,6_1,70_1,71_1,72_1,73_1,7_1,8_1,9_1
113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.153334,4.487399,8.154235,-1.037930,-0.873195,-1.471254,6.436117,-1.192968,-1.672527,-0.432370
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.489438,-17.350323,-2.118146,-1.383906,-1.164261,-6.200455,8.581490,-11.689613,-2.230036,-0.576494
261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.797958,4.135931,-1.588609,-1.037930,-0.873195,-0.514715,6.436117,5.146751,-1.672527,-0.432370
325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.661556,4.770714,-1.323841,-0.864941,-0.727663,-2.116940,5.363431,-0.587748,-1.393773,-0.360309
332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.462722,1.556818,-0.529536,-0.345977,-0.291065,-0.846776,2.145372,1.626916,-0.557509,-0.144123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4439680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.361546,0.778409,-0.264768,-0.172988,-0.145533,-0.423388,1.072686,-0.117550,-0.278755,-0.072062
4439823,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.287339,1.556818,-0.529536,-0.345977,-0.291065,0.503633,2.145372,0.733422,-0.557509,-0.144123
4439898,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.971922,6.887117,-2.647682,-1.729883,-1.455326,-3.516475,9.011221,-11.836044,-2.787545,-0.720617
4439949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.336008,0.540436,-0.264768,-0.172988,-0.145533,3.365258,-0.642955,0.080247,-0.278755,-0.072062


In [16]:
# session_train.to_csv("../datasets/train_ses_vec_x_992.csv")
# session_test.to_csv("../datasets/test_ses_vec_x_992.csv")

### turn testing candidate into vectors

In [27]:
test_purchase = pd.read_csv("../datasets/test_purchase.csv",index_col=0)
test_purchase.item_id.nunique()

4769

In [29]:
test_candidate_992 = sessions2vec(item_vec,test_purchase)
test_candidate_992.head()

total session:  81618  time usage:  28.328518629074097  sec


,10_147,10_159,10_184,10_217,10_22,10_287,10_361,10_407,10_464,10_561,...,68_1,69_1,6_1,70_1,71_1,72_1,73_1,7_1,8_1,9_1
113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.336008,0.686881,4.606654,-0.172988,-0.145533,0.458129,1.072686,1.260205,-0.278755,-0.072062
115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.336008,-0.341893,-0.264768,-0.172988,-0.145533,-0.423388,1.072686,-0.444937,-0.278755,-0.072062
261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.336008,0.778409,-0.264768,-0.172988,-0.145533,-0.423388,1.072686,0.080247,-0.278755,-0.072062
325,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.941106,-0.341893,-0.264768,-0.172988,-0.145533,-0.423388,1.072686,0.209838,-0.278755,-0.072062
332,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.133233,0.686881,0.269435,-0.172988,-0.145533,2.732254,1.072686,-1.454381,-0.278755,-0.072062


In [31]:
#test_candidate_992.to_csv("../datasets/test_candidate_992.csv")